In [172]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup

In [173]:
data = pd.read_csv('data/test_news.csv')

In [174]:
data.head()

,Unnamed: 0,NewsId,OrganCode,NewsCategoryCode,IcbCode,ComGroupCode,ExpertId,Ticker,NewsTitle,FriendlyTitle,...,NewsKeyword,FriendlyKeyword,PublicDate,IsTranfer,CreateBy,UpdateBy,Status,CreateDate,UpdateDate,SourceCode
0,0,4039,NaN,MAnal,OTHER,NaN,NaN,NaN,Hết phiên 1: Giảm mạnh nhưng không phá vỡ đáy cũ,NaN,...,NaN,NaN,NaN,True,NaN,NaN,1,2015-06-04 00:00:00,2015-06-04 00:00:00,NaN
1,1,4040,NaN,MAnal,OTHER,NaN,NaN,NaN,UBCKNN tiếp tục thực hiện nhiều biện pháp giám...,NaN,...,NaN,NaN,NaN,True,NaN,NaN,1,2015-06-04 00:00:00,2015-06-04 00:00:00,NaN
2,2,4041,NaN,Fina2,OTHER,NaN,NaN,NaN,Ngân hàng làm gì với hàng tỷ USD rút về nước?,NaN,...,NaN,NaN,NaN,True,NaN,NaN,1,2015-06-04 00:00:00,2015-06-04 00:00:00,NaN
3,3,4042,NaN,MAnal,OTHER,NaN,NaN,NaN,"Kết thúc ngày giao dịch: VN-Index giảm hơn 1,3%.",NaN,...,NaN,NaN,NaN,True,NaN,NaN,1,2015-06-04 00:00:00,2015-06-04 00:00:00,NaN
4,4,4043,NaN,MAnal,OTHER,NaN,NaN,NaN,VN-Index &quot;phá đáy&quot; trong xu thế giảm...,NaN,...,NaN,NaN,NaN,True,NaN,NaN,1,2015-06-04 00:00:00,2015-06-04 00:00:00,NaN


In [175]:
data.columns

Index(['Unnamed: 0', 'NewsId', 'OrganCode', 'NewsCategoryCode', 'IcbCode',
       'ComGroupCode', 'ExpertId', 'Ticker', 'NewsTitle', 'FriendlyTitle',
       'NewsSubTitle', 'FriendlySubTitle', 'NewsShortContent',
       'NewsFullContent', 'NewsImageUrl', 'NewsSmallImageUrl', 'NewsSource',
       'NewsSourceLink', 'NewsAuthor', 'NewsKeyword', 'FriendlyKeyword',
       'PublicDate', 'IsTranfer', 'CreateBy', 'UpdateBy', 'Status',
       'CreateDate', 'UpdateDate', 'SourceCode'],
      dtype='object')

In [176]:
data_process = data[['NewsId', 'Ticker', 
                       'NewsTitle', 'NewsFullContent', 'PublicDate']]

In [177]:
data_process.head(10)

,NewsId,Ticker,NewsTitle,NewsFullContent,PublicDate
0,4039,NaN,Hết phiên 1: Giảm mạnh nhưng không phá vỡ đáy cũ,<P>Với những lo ngại của các nhà đầu tư trong ...,NaN
1,4040,NaN,UBCKNN tiếp tục thực hiện nhiều biện pháp giám...,<DIV class=KenhF_Content_News3>\r\n<P></P>\r\n...,NaN
2,4041,NaN,Ngân hàng làm gì với hàng tỷ USD rút về nước?,"<P>Khủng hoảng tài chính lan rộng, các ngân hà...",NaN
3,4042,NaN,"Kết thúc ngày giao dịch: VN-Index giảm hơn 1,3%.","Sau phiên giao dịch đầy khởi sắc ngày hôm qua,...",NaN
4,4043,NaN,VN-Index &quot;phá đáy&quot; trong xu thế giảm...,<DIV class=div_fleft_p10_w250 id=ctl00_Content...,NaN
5,4044,NaN,Những khúc mắc quanh thuế thu nhập cá nhân với...,<DIV class=KenhF_Content_News3>\r\n<P class=Ms...,NaN
6,4045,NaN,"Chứng khoán ngày 23/10: Lập đáy mới, chờ tín hiệu",<P>Nhiều nhà đầu tư chờ khối ngoại mua ròng tr...,NaN
7,4046,NaN,Có hay không việc nhà đầu tư ngoại rút vốn?,<DIV class=KenhF_Content_News3>\r\n<P><SPAN st...,NaN
8,4047,NaN,Kết thúc ngày 24/10: VN-Index đã xuống gần mốc...,<P>Sau khi thủng đáy cũ 366 trong phiên giao d...,NaN
9,4048,NaN,Phố Wall ngóng chờ quyết định của FED,"<FONT face=""arial, helvetica, sans-serif""><FON...",NaN


In [178]:
# data_process = data_process.dropna(subset=['Ticker', 'PublicDate', 'NewsFullContent'])

In [179]:
data_process['NewsFullContent'][1000]

'<SPAN id=ctl00_mainContent_lblContent style="FONT-SIZE: 8.5pt; COLOR: black; FONT-FAMILY: Arial">\r\n<STYLE>\n<!--\n /* Style Definitions */\n p.MsoNormal, li.MsoNormal, div.MsoNormal\n\t{mso-style-parent:"";\n\tmargin:0in;\n\tmargin-bottom:.0001pt;\n\tmso-pagination:widow-orphan;\n\tfont-size:12.0pt;\n\tfont-family:"Times New Roman";\n\tmso-fareast-font-family:"Times New Roman";}\n@page Section1\n\t{size:595.45pt 841.7pt;\n\tmargin:.5in .75in 45.35pt 1.25in;\n\tmso-header-margin:.5in;\n\tmso-footer-margin:.5in;\n\tmso-paper-source:0;}\ndiv.Section1\n\t{page:Section1;}\n-->\n</STYLE>\r\n\r\n<BLOCKQUOTE><SPAN style="FONT-SIZE: 10pt; FONT-FAMILY: Arial"><O:P></O:P>Thay đổi nội dung Phương án phát hành thêm cổ phiếu năm 2008, cụ thể:<O:P></O:P></SPAN>\r\n<P class=MsoNormal><SPAN style="FONT-SIZE: 10pt; FONT-FAMILY: Arial">1. Đối tượng phát hành:<O:P></O:P></SPAN></P>\r\n<BLOCKQUOTE>\r\n<P class=MsoNormal><SPAN style="FONT-SIZE: 10pt; FONT-FAMILY: Arial">- Cổ đông hiện hữu: 1.443.737 cổ p

In [180]:
def cleanhtml(raw_html):
    for i in range(len(raw_html)):
        cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|\n|\t|\r|\xa0|&nbsp;')
        raw_html[i] = re.sub(cleanr,'', str(raw_html[i]))
        raw_html[i] = BeautifulSoup(raw_html[i], "lxml").text
    return raw_html

In [181]:
data_array_clean = cleanhtml(data_process['NewsFullContent'])
data_process['NewsFullContent'] = data_array_clean

In [187]:
data_process.shape

(2001, 5)

In [189]:
data_process.to_csv('data/data_preprocess.csv', encoding='utf-8')